# Gradient Descent

Gradient Descent is an algorithm that minimizes some objective $J(\theta)$ where $\theta$ is $\in \mathbb{R}^d$ by iteratively updating the vector $\theta$ by moving in the direction of steepest descent multiplied by some step size or learning rate.

In essence, 

$\>\>\>\>$1: Pick an initial guess $\theta_0$    
$\>\>\>\>$2: $\>\>$while $k \in [0...iteration_{max}]$ and $\theta_{k+1} - \theta_{k} \leq precision$:    
$\>\>\>\>$3: $\>\>\>\>\>\>\>\>$ $\theta_{k+1}$ = $\theta_{k} -\nabla J_\theta(\theta_k)$ * $\alpha$.

This is relevant to Machine Learning because it offers a numerical solution to estimates of parameters that do not have a closed form solution.

## Estimating Multinomial Logistic Regression Parameters with Gradient Descent 

However, one needs to determine the objective $J(\theta)$ before executing this algorithm. In the case of linear regression, the sum of squared residuals could be used. In this case, one can use the idea of MLE to minimize the log liklihood.

In the case where $K = 2$, the likelihood function was $\Pi_{i=1}^n p(\textbf x_i)^{y_i} (1-p(\textbf x_i))^{1-y_i}$ 

In the case where $K > 2$,

For $k \in K$ where $K$ is the set of labeled classes and $p$ is the number of features      
Let $p_k(\textbf x)$ be the posterior probability $P(Y = k | X = \bf x)$ and the model be defined as 

$p_k(\textbf x) = \frac{e^{\beta_{0}^k + \textbf w_k\cdot \textbf x}}{\sum_{k=1}^{K} e^{\beta_{0}^k + \textbf w_k\cdot \textbf x}}$ for $k=1...K$

One can see how each class $1 \leq k \leq K$ has its own set of parameters $w_k = (\beta_1,...,\beta_p)$

We want the optimal set of parameters(weights): argmax $L(\beta$ $|$ $ \textbf x)$

$$\prod_{i=1}^{N}\prod_{k=1}^{K}P(Y = k | X = \textbf x_i)^{k_i} = \prod_{i=1}^{N}\prod_{k=1}^{K} p_k(\textbf x_i)^{k_i}$$

Minimizing the negative log-liklihood is the same as maximizing the log-liklihood. 

$$J(\textbf w) = -log\prod_{i=1}^{N}\prod_{k=1}^{K} p_k(\textbf x_i)^{k_i} = -\sum_{i=1}^{N}\sum_{k=1}^{K} {k_i} * log(\hat{y}_{ki})$$

where $k_i$ is 1 if $\textbf x_i$ belongs to the class k

In [7]:
import numpy as np

In [13]:
class BatchGradientDescent:
    def _init_(self):
        self.rate = .01
        self.precision = .001
    
    def gradient(self):
        return 0
    
    def determineCost(self):
        return 0
    def gradientDescent(self):
        return 0

In [10]:
class LogisticRegression:
    
    def addBias(self):
        return 0
    
    def normalize(self):
        return 0
    
    def train(self,trainX,trainy):
        return 0

In [6]:
# loading the mnist dataset
from keras.datasets import mnist
from matplotlib import pyplot
# load dataset
(trainX, trainy), (testX, testy) = mnist.load_data()
# summarize loaded dataset
print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
print('Test: X=%s, y=%s' % (testX.shape, testy.shape))

Train: X=(60000, 28, 28), y=(60000,)
Test: X=(10000, 28, 28), y=(10000,)
